In [1]:
import requests
import time
import itertools
from collections import defaultdict

STEAM_API_URL = "https://store.steampowered.com/appreviews/220"

REVIEW_TYPES = ['positive', 'negative']
PURCHASE_TYPES = ['steam', 'non_steam_purchase']
LANGUAGES = ['english', 'german', 'french', 'spanish', 'russian', 'chinese', 'japanese', 'korean', 'brazilian', 'all']

def fetch_reviews(appid, review_type, purchase_type, language, num_per_request=100):
    cursor = '*'
    reviews = []
    seen_ids = set()
    
    while True:
        params = {
            'json': 1,
            'filter': 'all',
            'language': language,
            'review_type': review_type,
            'purchase_type': purchase_type,
            'num_per_page': num_per_request,
            'cursor': cursor
        }

        try:
            response = requests.get(STEAM_API_URL, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
        except Exception as e:
            print(f"Error: {e}")
            break

        batch = data.get("reviews", [])
        if not batch:
            break

        for review in batch:
            rid = review.get("recommendationid")
            if rid not in seen_ids:
                reviews.append(review)
                seen_ids.add(rid)

        old_cursor = cursor
        cursor = data.get("cursor")
        if not cursor or old_cursor == cursor:
            break

        time.sleep(0.2)  # respect API rate limits

    return reviews

def collect_all_reviews(appid):
    results = defaultdict(list)
    for rt, pt, lang in itertools.product(REVIEW_TYPES, PURCHASE_TYPES, LANGUAGES):
        key = f"{rt}_{pt}_{lang}"
        print(f"Fetching: {key}")
        reviews = fetch_reviews(appid, rt, pt, lang)
        results[key] = reviews
        print(f"Retrieved {len(reviews)} reviews for {key}")
    return results

# Run the collector for Half-Life 2 (appid: 220)
all_reviews = collect_all_reviews(appid=220)

# Example: Save to disk or filter for unique reviews later
# e.g., by recommendationid or Steam user ID


Fetching: positive_steam_english


KeyboardInterrupt: 